# Partie 3 : Spark (11 points)
## Objectf et utlité de notre traitement Spark

L'objectif de notre traitement Spark est de pouvoir identifier les tendances locales de location par type de logement.

Cela révèle les types de propriétés et les villes où les hôtes imposent des séjours minimums plus longs, ce qui peut être indicatif de régions avec une demande de location plus stable ou orientée vers des séjours prolongés (par exemple, pour des touristes ou des voyageurs d'affaires préférant des locations à long terme).



## Description des étapes de notre traitement Spark

### Initialisation de la session Spark :

Une session Spark est créée en utilisant SparkSession.builder, avec un nom d’application pour faciliter le suivi des processus dans l’interface de gestion de Spark.

### Chargement des données :

* Importer le fichier csv "listings_clean.csv" dans l'environnement de gogle colab.

* Les données sont chargées depuis un fichier CSV nommé "listings_clean.csv", avec header=True pour lire la première ligne comme en-tête et inferSchema=True pour déduire automatiquement les types de colonnes.

* L’option multiLine=True permet de traiter les champs de texte multiline, tandis que escape='"' et quote='"' garantissent une gestion correcte des guillemets dans les champs de texte.

### Affichage d’un échantillon des données :

Afficher les cinq premières lignes du DataFrame pour vérifier la structure et le contenu des données, et s’assurer que les colonnes importantes comme "ville", "property_type", "minimum_nights" et "maximum_nights" sont présentes et bien interprétées.

### Regroupement et agrégation par ville et type de propriété :

* Les données sont regroupées par ville et property_type (type de logement).

* Les moyennes des colonnes "minimum_nights" et "maximum_nights" sont calculées pour chaque groupe. Les résultats sont stockés dans de nouvelles colonnes appelées "moyenne_minimum_nights" et "moyenne_maximum_nights".

### Tri des résultats :

Les résultats sont triés en ordre décroissant selon la colonne "moyenne_minimum_nights", ce qui permet de visualiser en premier les villes et types de logements avec les séjours minimums moyens les plus élevés.

### Affichage des résultats agrégés :

Affichage des résultats finaux du traitement, montrant pour chaque ville et type de logement les moyennes des séjours minimum et maximum.

## Implémentation des étapes de notre traitement Spark

In [ ]:
!wget https://data.insideairbnb.com/canada/qc/montreal/2024-09-13/data/listings.csv.gz

In [19]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [20]:
# Initialisation de la session Spark
spark = SparkSession.builder.appName("Analyse des durées de séjour par ville et type de logement").getOrCreate()


In [21]:
# Chargement des données
data = spark.read.options(header=True, inferSchema=True, multiline=True, escape='"').csv("listings.csv.gz")

In [22]:
df.show(5)

+-----+--------------------+--------------+------------+-----------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+---------+----------+----------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+--------+---------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+----------------+---------------------+-----------

In [24]:
# Nettoyage des données (suppression des valeurs non valides dans les colonnes minimum_nights et maximum_nights)
df_cleaned = (
    data
    .withColumns(
        {
          "ville": F.split(F.col("host_location"), ", ").getItem(0),
          "pays": F.split(F.col("host_location"), ", ").getItem(1)
        }
    )
    .where((F.col("pays") == F.lit("Canada")) & (F.col("minimum_nights").isNotNull()) & (F.col("maximum_nights").isNotNull()))
    .select(
        F.col("name"),
        F.col("description"),
        F.col("neighborhood_overview"),
        F.col("host_name"),
        F.col("host_since"),
        F.col("ville"),
        F.col("property_type"),
        F.col("room_type"),
        F.col("accommodates"),
        F.col("bathrooms"),
        F.col("bathrooms_text"),
        F.col("bedrooms"),
        F.col("beds"),
        F.col("amenities"),
        F.col("price"),
        F.col("minimum_nights"),
        F.col("maximum_nights"),
        F.col("number_of_reviews"),
        F.col("review_scores_rating"),
        F.col("last_review")
    )
)

In [25]:
df_cleaned.show()

+--------------------+--------------------+---------------------+---------------+----------+--------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+-----------------+--------------------+-----------+
|                name|         description|neighborhood_overview|      host_name|host_since|   ville|       property_type|      room_type|accommodates|bathrooms|bathrooms_text|bedrooms|beds|           amenities|  price|minimum_nights|maximum_nights|number_of_reviews|review_scores_rating|last_review|
+--------------------+--------------------+---------------------+---------------+----------+--------+--------------------+---------------+------------+---------+--------------+--------+----+--------------------+-------+--------------+--------------+-----------------+--------------------+-----------+
|Fabulous downtown...|Come stay in this...| This area of St H...|           Gail|2008-08-19|Montr

In [27]:
# Regroupement par ville et type de propriété, et calcul des moyennes de nuitées
df_grouped = df_cleaned.groupBy("ville","property_type").agg(
    F.avg("minimum_nights").alias("moyenne_minimum_nights"),
    F.avg("maximum_nights").alias("moyenne_maximum_nights")
).orderBy(F.col("moyenne_minimum_nights").desc())


In [28]:
# Affichage des résultats
df_grouped.show()

+----------------+--------------------+----------------------+----------------------+
|           ville|       property_type|moyenne_minimum_nights|moyenne_maximum_nights|
+----------------+--------------------+----------------------+----------------------+
|         Candiac|  Entire rental unit|                 365.0|                 365.0|
|  Mont-Tremblant|Private room in r...|                 365.0|                 365.0|
|          Quebec|        Entire condo|                 198.0|                 632.5|
|       Longueuil|         Entire loft|                 185.0|                1125.0|
|       Dartmouth|        Entire condo|                 182.0|                 365.0|
|  Deux-Montagnes|  Entire rental unit|                 180.0|                1125.0|
|        Kirkland|         Entire loft|                 180.0|                1125.0|
|        Brossard|        Entire condo|                 180.0|                1125.0|
|         Calgary|  Entire rental unit|               